# NN Chatbot Training 

### Trains the conversational attributes of our chatbot
<hr>

In [15]:
"""


"""

'\npython=3.7\ndid pip\ntensorflow==2.3.1\nnltk==3.5\ncolorama==0.4.3\nnumpy==1.18.5\nscikit_learn==0.23.2\nFlask==1.1.2\n\n'

<br>

# Imports
<hr>

In [72]:
# Import

import json 
import numpy as np 

import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Var Declaration

In [ ]:
# 
training_sentences = []
training_labels = []
labels = []
responses = []

#
vocab_size = 1000
embedding_dim = 16
max_len = 20 
oov_token = '<00V>'

<br>

# Data Ingestion
<hr>

In [34]:
"""
    This code block opens the json file and enters the data into arrays (lists),
    so that it can be feed into the pipeline. 

"""

# Open json file
with open('../data/intents.json') as file:
    data = json.load(file)



# loop to load data 
for intent in data['intents']:
    for pattern in intent['patterns']:
        #
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    #
    responses.append(intent['responses'])

    # creates labels list
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


num_classes = len(labels)

In [52]:
# inspect Data

print(f'Training Sentences: {training_sentences[:7]}')
print(f'Training Labels:    {training_labels[:6]}')
print(f'Labels:             {labels[:7]}')
print(f'Responses:          {responses[:3]}')

Training Sentences: ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later']
Training Labels:    ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye']
Labels:             ['greeting', 'goodbye', 'thanks', 'about', 'name', 'help', 'createaccount']
Responses:          [['Hello', 'Hi', 'Hi there'], ['See you later', 'Have a nice day', 'Bye! Come back again'], ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"]]


<br> 

# Preprocessing
<hr>

##### Normalize Labels w/LabelEncoder 

In [31]:

# define model 
lbl_encoder = LabelEncoder()
# fit model 
lbl_encoder.fit(training_labels)
# transform model 
training_labels = lbl_encoder.transform(training_labels)



In [32]:
# View Encoded Labels

training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])


##### Normalize Tokenizer Data

In [19]:
"""
 Tokenize data
 
 setting number of words to 1,000
 Using oov_token 

"""

# declare tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
# fit tokenier
tokenizer.fit_on_texts(training_sentences)
# creates dictonary of words
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
# creates equal sizes between the text sequences
padded_sequences = pad_sequences(sequences, truncating = 'post', maxlen = max_len)


In [71]:
# inspect tokenized var

print()
print(f'Len of word_index: {len(word_index)}')
print(f'Sequences:         {sequences[:7]} ')
print()
print('padded Sequences: ')
padded_sequences[:3]


Len of word_index: 52
Sequences:         [[29], [30], [10, 31, 18], [32], [33], [34], [35, 2, 36]] 

padded Sequences: 


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 29],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 10, 31, 18]], dtype=int32)

In [20]:
# define model 

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation = 'softmax'))


In [21]:
# Compile model

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [22]:
# Model Summory 

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dense_6 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [23]:
# Train Model 

epochs = 279
history = model.fit(padded_sequences, np.array(training_labels),  epochs=epochs)

Epoch 1/500
2/2 [==============================] - 2s 13ms/step - loss: 2.0792 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - 0s 19ms/step - loss: 2.0778 - accuracy: 0.0909
Epoch 3/500
2/2 [==============================] - 0s 11ms/step - loss: 2.0773 - accuracy: 0.0606
Epoch 4/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0767 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0758 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0752 - accuracy: 0.1515
Epoch 7/500
2/2 [==============================] - 0s 8ms/step - loss: 2.0747 - accuracy: 0.2424
Epoch 8/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0741 - accuracy: 0.4848
Epoch 9/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0736 - accuracy: 0.2727
Epoch 10/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0732 - accuracy: 0.2424
Epoch 11/500
2/2 [=======

# Making a Pickle
<hr>

In [24]:
# save the trained model 
model.save('../assets/chat_model')

# save the fitted Tokenizer
with open('../assets/tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fittend label encoder
with open('../assets/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


    

INFO:tensorflow:Assets written to: ../assets/chat_model/assets
